In [ ]:
import pandas as pd



data_path = "C:/Users/annam/Desktop/ITU/2nd_sem/02_First_Year_Project/2nd_project/Data/full_data.csv"
df = pd.read_csv(data_path)
df


In [ ]:
#new_frame = pd.DataFrame(data = df["img_id"], columns = df["diagnostic"])
#new_frame

new_df=df[['img_id','diagnostic']]
new_df.loc[new_df['diagnostic'].isin(['BCC', 'MEL', 'SCC']), 'diagnostic'] = int(1)
new_df.loc[new_df['diagnostic'].isin(['ACK', 'NEV', 'SEK']), 'diagnostic'] = int(0)


In [98]:
#Feature extraction

import functions
import os
from skimage import io

folder_path_in = "C:/Users/annam/Desktop/Color_mask/Color_mask/Training"
def extract_features(folder_path_in):
    # Create a list of image filenames
    feature_1 = []
    feature_2 = []
    feature_3 = []
    feature_4 = []
    feature_5 = []
    feature_6 = []
    feature_7 = []
    #Iterate through all the jpg and png files in the selected folder
    for filename in [f for f in os.listdir(folder_path_in) if f.endswith('.jpg') or f.endswith('.png')]:
        
        #Read in the image
        image_path = folder_path_in + "/" + filename
        original = io.imread(image_path)

        # Ignore the alpha channel (e.g. transparency )
        if original.shape[-1] == 4:
            original = original[..., :3]
        
        feature_1.append(functions.measure_pigment_network(original))
        feature_2.append(functions.measure_blue_veil(original))
        feature_3.append(functions.measure_vascular(original))
        feature_4.append(functions.measure_globules(original))
        feature_5.append(functions.measure_streaks(original))
        feature_6.append(functions.measure_irregular_pigmentation(original))
        feature_7.append(functions.measure_regression(original))
    return feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7

   
   

In [100]:
file_names = []
features_df = pd.DataFrame()
for filename in os.listdir(folder_path_in):
    if filename.endswith('.jpg') or filename.endswith('.png') or filename.endswith('.jpeg'):
        file_names.append(filename)

features_df = pd.DataFrame({'img_id': file_names})

def featurized_df(feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7):
   


    features_df["1: pigment network"] = feature_1
    features_df["2: Blue veil"] = feature_2
    features_df["3: Vascular"] = feature_3
    features_df["4: Globules"] = feature_4
    features_df["5: Streaks"] = feature_5
    features_df["6: Pigmentation"] = feature_6
    features_df["7: Regression"] = feature_7

    return features_df



In [101]:
import pandas as pd

# Assuming df1 and df2 are your DataFrames
# Assuming 'img_id' is the column name containing the image IDs

# Check if the img_id values are the same in both DataFrames
df_merged = pd.merge(new_df, features_df, on='img_id', how='inner')

# Create a new column called 'diagnosed' based on the comparison
#df_merged['diagnosed'] = True

# If you want to fill the 'diagnosed' column with False for non-matching img_id values, uncomment the following line:
#df_merged['diagnosed'].fillna(False, inplace=True)

# Print the resulting DataFrame


In [102]:
from sklearn.model_selection import train_test_split

X = df_merged.copy()
X.drop("img_id", axis = 1, inplace = True)
X.drop("diagnostic", axis = 1, inplace = True)
Y = df_merged["diagnostic"].astype(int)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .5)





In [103]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier()

#Training the classifier
classifier = classifier.fit(X_train, Y_train)

prediction = classifier.predict(X_test)

ValueError: at least one array or dtype is required

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score
cm = confusion_matrix(Y_test, prediction, labels = [0, 1])
cm

In [ ]:
from sklearn import tree
from matplotlib import pyplot as plt
feature_names = X.columns
fig = plt.figure(figsize = (8,8))
_ = tree.plot_tree(classifier, feature_names= feature_names, class_names= {0:"cancer",1: "no cancer"})


In [ ]:
from sklearn.linear_model import LogisticRegression

cla = LogisticRegression()

cla = cla.fit(X_train, Y_train)
prediction = classifier.predict(X_test)
cm = confusion_matrix(Y_test, prediction, labels = [0, 1])
cm

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

classif = KNeighborsClassifier(n_neighbors = 5)

classif = cla.fit(X_train, Y_train)
pred = classifier.predict(X_test)
conf = confusion_matrix(Y_test, pred, labels = [0, 1])
conf

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, f1_score, roc_auc_score

# Perform cross-validation for Linear Regression
linear_regression_scores = cross_val_score(cla, X_train, Y_train, cv=5)
linear_regression_mean_accuracy = linear_regression_scores.mean()

# Perform cross-validation for KNN
knn_scores = cross_val_score(classif, X_train, Y_train, cv=5)
knn_mean_accuracy = knn_scores.mean()

# Perform cross-validation for Decision Tree
tree_scores = cross_val_score(classifier, X_train, Y_train, cv=5)
tree_mean_accuracy = tree_scores.mean()

# Calculate F1 score for Linear Regression
y_pred_linear_regression = cla.predict(X_test)
y_pred_linear_regression = [0 if val < 0.5 else 1 for val in y_pred_linear_regression]
f1_linear_regression = f1_score(Y_test, y_pred_linear_regression)

# Calculate F1 score for KNN
y_pred_knn = classif.predict(X_test)
f1_knn = f1_score(Y_test, y_pred_knn)

# Calculate F1 score for Tree
y_pred_tree = classifier.predict(X_test)
f1_tree = f1_score(Y_test, y_pred_tree)

# Calculate AUC-ROC score for Linear Regression
auc_roc_linear_regression = roc_auc_score(Y_test, y_pred_linear_regression)

# Calculate AUC-ROC score for KNN
auc_roc_knn = roc_auc_score(Y_test, y_pred_knn)

# Calculate AUC-ROC score for KNN
auc_roc_tree = roc_auc_score(Y_test, y_pred_tree)

# Print the evaluation metrics
print("Linear Regression Cross-Validation Accuracy:", linear_regression_mean_accuracy)
print("KNN Cross-Validation Accuracy:", knn_mean_accuracy)
print("Tree Cross-Validation Accuracy:", tree_mean_accuracy)
print("Linear Regression F1 Score:", f1_linear_regression)
print("KNN F1 Score:", f1_knn)
print("Tree F1 Score:", f1_tree)
print("Linear Regression AUC-ROC Score:", auc_roc_linear_regression)
print("KNN AUC-ROC Score:", auc_roc_knn)
print("Tree AUC-ROC Score:", auc_roc_tree)

In [96]:
#Saving the classifier
import pickle
pickle.dump(classif, open("C:/Users/annam/Desktop/Trained_classifiers/KNN_2.pkl", "wb"))

In [97]:
import pickle

# Load the trained model from the saved file
with open("C:/Users/annam/Desktop/Trained_classifiers/KNN_2.pkl", 'rb') as file:
    model = pickle.load(file)

# Assuming you have a new image to predict
image_path = "C:/Users/annam/Desktop/Vascular/Masked"


# Extract features from the preprocessed image
image_features = extract_features(image_path)

# Make the prediction using the model
prediction = model.predict(image_features)

# Convert the numerical prediction back to the corresponding label
label_mapping = {0: 'benign', 1: 'malignant'}
predicted_label = label_mapping[prediction[0]]

# Print the predicted label
print("Predicted Label:", predicted_label)



0


c:\Users\annam\Desktop\ITU\2nd_sem\02_First_Year_Project\2nd_project\Project-2-Medical-Imaging\src\functions.py:160: RuntimeWarning: divide by zero encountered in scalar divide
  circularity = 4 * 3.1415 * (area / (perimeter ** 2))


0
0
0
1
2
1
0


c:\Users\annam\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


ValueError: X has 8 features, but KNeighborsClassifier is expecting 7 features as input.

In [ ]:
KNN_model = pickle.load(open("C:/Users/annam/Desktop/Trained_classifiers/KNN.pkl", "rb"))
KNN_model.predict(io.imread("C:/Users/annam/Desktop/Vascular/Masked/image5.png"))